In [32]:
import instructor
from openai import OpenAI, AsyncOpenAI
from pydantic import BaseModel
import numpy as np


In [5]:
from dotenv import load_dotenv
import os

# Load environment variables from a .env file
load_dotenv("../.envrc")


True

In [6]:
# Patch the OpenAI client
openai_client = OpenAI()
client = instructor.from_openai(AsyncOpenAI())


### Load documents


In [12]:
! realpath ~/Downloads/aubford/


/Users/shahules/Downloads/aubford


In [13]:
import os

# Define the folder path
folder_path = "/Users/shahules/Downloads/aubford"

# List all files in the folder
files = os.listdir(folder_path)

# Read each text file in the folder
texts = []
for file_name in files:
    if file_name.endswith(".txt"):
        with open(os.path.join(folder_path, file_name), "r") as file:
            texts.append(file.read())


In [14]:
texts[0]


'## Pepwave Forum Topic: Starlink and Peplink Balance\n\nI have recently joined Starlink\'s beta program.  They don\'t disclose any of their networking details.  They allow you to hook into their wifi router with a network out, but I found it was both unstable and had poor performance.\n\nI have managed to connect the Starlink dish to one of my WAN ports in DHCP/NAT mode, with an automatic DNS.  I have a second provider with the same settings.\n\nIt works great, however in order to use the management app, I believe need to be able to pass through 192.168.100.1 to WAN1 to allow a LAN client to manage the dish.  I have tried just about all of the Peplink balance settings to do so, an outbound policy to all WANs, adding 192.168.100.1 as an additional IP and pushing all traffic to that IP as a NAT rule, adding a firewall rule, and an management admin.  I was trying to get this 192.168.100.1 IP setup to pass through to the WAN1 gateway, but didn\'t manage to get that working either.\n\nI ca

## Generate personas


In [21]:
class Persona(BaseModel):
    role: str
    description: str


async def get_persona(text: str):
    prompt = f"""
    Given the following text, write a persona that may interact with it.
    Text: {text}
    """

    resp = await client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=Persona,
        messages=[
            {"role": "system", "content": "Your are expert evaluator"},
            {"role": "user", "content": prompt},
        ],
    )
    return resp


In [22]:
await get_persona(text=texts[0])


Persona(role='Technical Network Engineer', description='A professional specializing in network configurations and troubleshooting, particularly with experience in WAN setups and integrating satellite internet systems like Starlink with networking hardware such as Peplink Balance and Juniper SRX. This individual actively participates in technical forums to share knowledge, solve problems, and seek advice on complex network-related issues.')

### Generate a query per text


In [33]:
class Query(BaseModel):
    question: str


class Input(BaseModel):
    text: str
    persona: Persona


async def generate_query(text: str):

    persona = await get_persona(text=text)
    query_length = np.random.choice(["short", "medium", "long"])

    prompt = f"""
    Given the following text and a persona, write a query that the persona may ask.
    Persona: {persona}
    query length: {query_length}
    Text: {text}
    """

    resp = await client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=Query,
        messages=[
            {
                "role": "system",
                "content": "Imagine you're the persona described by the user.",
            },
            {"role": "user", "content": prompt},
        ],
    )
    return resp


## Test generations


In [34]:
await generate_query(text=texts[1])


Query(question='What are the recommended outbound policy settings for a Peplink Balance 20 to optimize internet connectivity?')